### Stpe0: Import library and Setting
- Import necessary library
- download NLTK resources

### Step 1: Data Loading and Preprocessing
- Load the dataset
- Split into training and test sets
- Explore the data distribution
- Apply text preprocessing (optional: remove stopwords, lowercase, etc.)

### Step 2: TF-IDF Feature Extraction
- Convert text documents into numerical TF-IDF features
- Understand how TF-IDF weights terms by importance
- Examine the vocabulary size and sample features
- Visualize the TF-IDF matrix structure

### Step 3: Baseline Model - Naive Bayes with TF-IDF
**YOU WILL IMPLEMENT:**

- Train a Naive Bayes classifier on the TF-IDF features
- Make predictions on the test set
- Create and visualize a confusion matrix to analyze:
    - True Positives (TP)
    - True Negatives (TN)
    - False Positives (FP)
    - False Negatives (FN)
- Calculate accuracy, precision, recall, and F1-score
- Interpret the results: What types of errors is the model making?

### Step 4: Dimensionality Reduction with SVD
- Apply Truncated SVD (Latent Semantic Analysis) to reduce TF-IDF dimensions
- Reduce from ~500-5000 features to 50-100 components
- Visualize the explained variance ratio
- Explain in your markdowns how SVD captures semantic relationships in text, and what it means for your data.


### Step 5: Model Training - Logistic Regression with SVD
**YOU WILL IMPLEMENT:**

- Train a Logistic Regression classifier on SVD-reduced features
- Make predictions on the test set
- Create and visualize a confusion matrix for the SVD model
- Compare performance with the baseline Naive Bayes model
- Analyze: Did dimensionality reduction help or hurt performance?
- Discuss on the markdowns: How does reducing dimensions affect model accuracy and speed?


### Step 6: Dimensionality Reduction with PCA
- Apply Principal Component Analysis (PCA) to the TF-IDF features
- First standardize the data (required for PCA)
- Reduce to the same number of components as SVD for fair comparison
- Compare PCA vs SVD variance curves


### Step 7: Model Training - Logistic Regression with PCA
**YOU WILL IMPLEMENT:**

- Train a Logistic Regression classifier on PCA-reduced features
- Make predictions on the test set
- Create and visualize a confusion matrix for the PCA model
- Compare performance with both previous models
- Analyze: Which dimensionality reduction technique works better for text data?


### Step 8: Visual Comparison
- Visualize all three confusion matrices side-by-side